In [1]:
from __future__ import division
from __future__ import print_function
import pandas as pd
from sklearn.model_selection import train_test_split

from pyod.models.iforest import IForest
from pyod.utils.data import generate_data
import numpy as np
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize

In [2]:
df=pd.read_csv(r'train.csv',index_col='PassengerId')


In [3]:
df=df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Survived']]

In [4]:
for colname in df.select_dtypes("object"):
    df[colname], _ = df[colname].factorize()


In [5]:
df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
PassengerId,,,,,,,,
1,3,0,22.0,1,0,7.2500,0,0
2,1,1,38.0,1,0,71.2833,1,1
3,3,1,26.0,0,0,7.9250,0,1
4,1,1,35.0,1,0,53.1000,0,1
5,3,0,35.0,0,0,8.0500,0,0
...,...,...,...,...,...,...,...,...
887,2,0,27.0,0,0,13.0000,0,0
888,1,1,19.0,0,0,30.0000,0,1
889,3,1,NaN,1,2,23.4500,0,0


In [21]:
df['Age']=df['Age'].fillna(df['Age'].mean())


In [22]:
X=df.drop('Survived',axis=1)
y=df['Survived']



In [23]:
contamination = 0.1  # percentage of outliers
n_train = 200  # number of training points
n_test = 100  # number of testing points

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22) 
# # Generate sample data
# X_train, X_test, y_train, y_test = \
#     generate_data(n_train=n_train,
#                   n_test=n_test,
#                   n_features=2,
#                   contamination=contamination,
#                   random_state=42)

In [25]:
X_train=X_train.to_numpy()
X_test=X_test.to_numpy()
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [26]:
clf_name = 'IForest'
clf = IForest()
clf.fit(X_train)
# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores
# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores
# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)
# example of the feature importance
feature_importance = clf.feature_importances_
print("Feature importance", feature_importance)
# visualize the results



On Training Data:
IForest ROC:0.7075, precision @ rank n:0.5556

On Test Data:
IForest ROC:0.7421, precision @ rank n:0.6222
Feature importance [0.07067296 0.07583299 0.26057272 0.13288335 0.12941239 0.25218669
 0.0784389 ]


In [29]:
y_test_scores.shape

(223,)